# Initialize

In [ ]:
!pip install datasets
!pip install transformers
import pandas as pd
import numpy as np
from datasets import load_dataset
import json
import torch
import torch.nn as nn
from typing import List, Tuple, Dict
import spacy
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import ast
import re

# Import Data and Exploration

In [ ]:
# Load csw dataset
csw_ds = load_dataset("CAiRE/ASCEND")

README.md:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/317M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/367M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/328M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9869 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1315 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1130 [00:00<?, ? examples/s]

In [ ]:
csw_translated = load_dataset("RuishiCh0314/ASCEND-mixed-to-chinese-translation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/672 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/417M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2739 [00:00<?, ? examples/s]

In [ ]:
with open("/content/Final_data_alpaca_standardized_data_data_class_finalized_csv_to_json_.json", 'r') as f:
        data = json.load(f)
cs_lan_zh = pd.DataFrame(data)

In [ ]:
with open("data3_chi_complex_finalized.json", 'rb') as f:
    data = json.load(f)
    # raw_data = f.read()
    # data = json.loads()  # or try other encodings
#raw_data.decode('utf-8-sig')
data3 = pd.DataFrame(data)

In [ ]:
with open("/content/data2_chi_complex_finalized.json", 'r', encoding='utf-8', errors='ignore') as f:
    raw_data = f.read()

data = json.loads(raw_data)
cs_lan_zh3 = pd.DataFrame(data)
cs_lan_zh3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51155 entries, 0 to 51154
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  51155 non-null  object
 1   input        51155 non-null  object
 2   output       51155 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


# Parallel Corpus Development

In [ ]:
# Load translation model to create parallel dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def ParallelCorpusGenerator(model="Helsinki-NLP/opus-mt-zh-en"):
  tokenizer = AutoTokenizer.from_pretrained(model)
  model = AutoModelForSeq2SeqLM.from_pretrained(model)

In [ ]:
def translate_dataframe(df, source_cols=['instruction', 'input'], batch_size=8):
    """
    Translate specified columns of a dataframe from Chinese to English

    Args:
        df: Input dataframe containing Chinese text
        source_cols: List of column names to translate
        batch_size: Number of texts to translate at once

    Returns:
        Parallel dataframe with original Chinese and translated English
    """
    # Initialize translator
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
    model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    def batch_translate(texts):
        """Translate a batch of texts"""
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs)

        return tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Create new dataframe for translations
    df_translated = pd.DataFrame()

    # Process each column
    for col in source_cols:
        if col not in df.columns:
            continue

        # Get all texts from the column
        texts = df[col].tolist()
        translations = []

        # Process in batches
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Translating {col}"):
            batch_texts = texts[i:i + batch_size]
            batch_translations = batch_translate(batch_texts)
            translations.extend(batch_translations)

        # Add both original and translated columns
        df_translated[f"{col}_zh"] = texts
        df_translated[f"{col}_en"] = translations

    return df_translated

In [ ]:
cs_lan_zh_parallel = translate_dataframe(cs_lan_zh3)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Translating input: 100%|██████████| 6395/6395 [40:20<00:00,  2.64it/s]


In [ ]:
cs_lan_en = translate_dataframe(cs_lan_zh)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Translating input: 100%|██████████| 93/93 [03:03<00:00,  1.98s/it]


# CSW Generation

## Process Aligned Texts

In [ ]:
aligned_df = pd.read_csv('/content/aligned_zh_en.csv')

In [ ]:
def process_alignments(df):
    def get_alignment_dict(info):
        try:
            # Get all alignment pairs
            align_list = eval(info)['alignment']
            result = {}
            for src, tgt in align_list:
                src_str = str(src)
                if src_str not in result:
                    result[src_str] = [tgt]
                else:
                    result[src_str].append(tgt)

            return result
        except:
            return None

    # Create new column with the processed alignments
    df['alignment_dict'] = df['alignment_info'].apply(get_alignment_dict)
    return df

In [ ]:
aligned_df = process_alignments(aligned_df)
print(aligned_df['alignment_dict'].head())

0          {'0': [0], '1': [1, 0], '2': [2], '3': [2]}
1    {'3': [4], '0': [6], '2': [3], '5': [0], '6': ...
2    {'0': [4, 5], '7': [7], '2': [7], '5': [1], '4...
3    {'2': [1], '0': [0], '5': [3, 2], '1': [0], '3...
4    {'4': [3], '0': [0], '3': [3], '2': [2], '1': ...
Name: alignment_dict, dtype: object


In [ ]:
# Check samples
index_of_waicun = aligned_df[aligned_df['instruction_zh'].str.contains('外存')].index.tolist()
print(index_of_waicun)
aligned_df.loc[9]

[9, 31]


,9
instruction_zh,什么是外存？
instruction_en,What's a memory?
input_zh,NaN
input_en,I don't think so.
alignment_info,"{'zh_words': ['什么', '是', '外存', '？'], 'en_words..."
alignment_dict,"{'0': [0], '1': [1, 0], '2': [2], '3': [2]}"


## Feature Extraction & First-round Synthetic CSW generation,

In [ ]:
import spacy
import jieba
from typing import List, Dict, Tuple
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
class FeatureExtractor:
    def __init__(self):
        self.en_nlp = spacy.load("en_core_web_sm")
        self.ner_pipeline = pipeline(
            "ner",
            model="dbmdz/bert-large-cased-finetuned-conll03-english",
            tokenizer="bert-base-cased",
            aggregation_strategy="simple"
        )

    def extract_features_from_english(self, text: str) -> Dict:
        """Extract linguistic features from English text"""
        doc = self.en_nlp(text)

        features = {
            'named_entities': [],
            'adjectives': [],
            'noun_phrases': [],
            'pos_tags': []
        }

        # Extract Named Entities
        try:
            ner_results = self.ner_pipeline(text)
            for ent in ner_results:
                if ent['entity_group'] in ['PER', 'ORG', 'LOC']:
                    features['named_entities'].append({
                        'text': ent['word'],
                        'token_indices': self.get_token_indices(doc, ent['word'])
                    })
        except Exception as e:
            print(f"NER extraction error: {e}")

        # Extract Adjectives and POS tags
        for token in doc:
            if token.pos_ == "ADJ":
                features['adjectives'].append({
                    'text': token.text,
                    'token_indices': [token.i]
                })
            features['pos_tags'].append({
                'text': token.text,
                'pos': token.pos_,
                'token_indices': [token.i]
            })

        # Extract Noun Phrases
        for chunk in doc.noun_chunks:
            features['noun_phrases'].append({
                'text': chunk.text,
                'token_indices': [token.i for token in chunk]
            })

        return features

    def get_token_indices(self, doc, text: str) -> List[int]:
        """Get token indices for a given text span"""
        indices = []
        text_lower = text.lower()
        for i, token in enumerate(doc):
            if token.text.lower() in text_lower:
                indices.append(i)
        return indices

    def output_pos_dataframe(self, df: pd.DataFrame, text_column: str) -> pd.DataFrame:
        """
        Process a DataFrame of texts and output a DataFrame of POS tags and other features.
        Args:
            df: The input DataFrame containing English text data.
            text_column: The column name containing the text to process.

        Returns:
            A DataFrame of POS tags and features.
        """
        data = []
        for _, row in df.iterrows():
            text = row[text_column]
            if isinstance(text, str):  # Ensure we only process textual data
                features = self.extract_features_from_english(text)
                for pos in features['pos_tags']:
                    data.append({
                        "original_text": text,
                        "token": pos['text'],
                        "pos": pos['pos'],
                        "token_indices": pos['token_indices']
                    })

        # Convert to DataFrame
        pos_df = pd.DataFrame(data)
        return pos_df

class CodeMixedGenerator:
    def __init__(self, feature_extractor: FeatureExtractor):
        self.feature_extractor = feature_extractor
        self.target_cmi_min = 0.20  # 20%
        self.target_cmi_max = 0.30  # 30%

    def calculate_cmi(self, tokens: List[str]) -> float:
        """Calculate Code-Mixing Index (CMI)"""
        if not tokens:
            return 0.0

        english_tokens = sum(1 for token in tokens if any(c.isascii() for c in token))
        total_tokens = len(tokens)
        return english_tokens / total_tokens if total_tokens > 0 else 0.0

    def should_replace_token(self, token_idx: int, features: Dict, current_cmi: float) -> bool:
        """Determine if a token should be replaced based on linguistic features and CMI"""
        # If CMI is already too high, don't replace
        if current_cmi > self.target_cmi_max:
            return False

        # If CMI is too low, be more aggressive with replacements
        if current_cmi < self.target_cmi_min:
            # Check if token is part of any feature
            for feature_list in [features['named_entities'], features['adjectives'], features['noun_phrases']]:
                for feature in feature_list:
                    if token_idx in feature['token_indices']:
                        return True
            return False

        # If CMI is in target range, be more selective
        for ne in features['named_entities']:
            if token_idx in ne['token_indices']:
                return True

        # Only replace adjectives and noun phrases if CMI is below target
        if current_cmi < self.target_cmi_min:
            for adj in features['adjectives']:
                if token_idx in adj['token_indices']:
                    return True

            for np in features['noun_phrases']:
                if token_idx in np['token_indices']:
                    return True

        return False

    def generate_code_mixed(self, en_text: str, zh_text: str, alignment_dict: str) -> str:
        try:
            features = self.feature_extractor.extract_features_from_english(en_text)

            if isinstance(alignment_dict, str):
                alignment_dict = ast.literal_eval(alignment_dict)

            en_tokens = en_text.split()
            zh_tokens = list(jieba.cut(zh_text))
            mixed_tokens = zh_tokens.copy()
            replaced_positions = set()

            # Initial CMI
            current_cmi = self.calculate_cmi(mixed_tokens)

            # Sort alignments by feature importance
            sorted_alignments = []
            for zh_pos, en_pos in alignment_dict.items():
                zh_idx = int(zh_pos)
                en_indices = en_pos if isinstance(en_pos, list) else [en_pos]

                # Calculate importance score based on features
                importance = 0
                for en_idx in en_indices:
                    if isinstance(en_idx, int) and en_idx < len(en_tokens):
                        if any(ne['token_indices'] == [en_idx] for ne in features['named_entities']):
                            importance += 3  # Highest priority for named entities
                        elif any(adj['token_indices'] == [en_idx] for adj in features['adjectives']):
                            importance += 2  # Medium priority for adjectives
                        elif any(en_idx in np['token_indices'] for np in features['noun_phrases']):
                            importance += 1  # Lower priority for noun phrases

                sorted_alignments.append((zh_idx, en_indices, importance))

            # Sort by importance score
            sorted_alignments.sort(key=lambda x: x[2], reverse=True)

            # Process alignments with CMI control
            for zh_idx, en_indices, _ in sorted_alignments:
                if zh_idx < len(mixed_tokens) and zh_idx not in replaced_positions:
                    current_cmi = self.calculate_cmi(mixed_tokens)

                    if current_cmi > self.target_cmi_max:
                        break  # Stop if we've exceeded maximum CMI

                    replacements = []
                    for en_idx in en_indices:
                        if isinstance(en_idx, int) and en_idx < len(en_tokens):
                            if self.should_replace_token(en_idx, features, current_cmi):
                                replacements.append(en_tokens[en_idx])

                    if replacements and current_cmi < self.target_cmi_max:
                        replacement = max(replacements, key=len)
                        mixed_tokens[zh_idx] = replacement
                        replaced_positions.add(zh_idx)

            # Post-processing (same as before)
            final_tokens = []
            for i, token in enumerate(mixed_tokens):
                if i > 0 and token == mixed_tokens[i - 1]:
                    continue
                final_tokens.append(token)

            cleaned_tokens = []
            for token in final_tokens:
                if any(c.isascii() for c in token):
                    if not (token.startswith("(") and token.endswith(")")):
                        token = token.lower()
                    token = re.sub(r"\bwhat's\b", "what", token)
                cleaned_tokens.append(token)

            # Final CMI check
            final_cmi = self.calculate_cmi(cleaned_tokens)
            if final_cmi < self.target_cmi_min:
                # If CMI is too low, try to add more English tokens
                for zh_idx, en_indices, _ in sorted_alignments:
                    if zh_idx < len(mixed_tokens) and zh_idx not in replaced_positions:
                        for en_idx in en_indices:
                            if isinstance(en_idx, int) and en_idx < len(en_tokens):
                                mixed_tokens[zh_idx] = en_tokens[en_idx]
                                replaced_positions.add(zh_idx)
                                if self.calculate_cmi(mixed_tokens) >= self.target_cmi_min:
                                    break
                        if self.calculate_cmi(mixed_tokens) >= self.target_cmi_min:
                            break

            # Join tokens with proper spacing
            result = ''
            for i, token in enumerate(cleaned_tokens):
                if (any(c.isascii() for c in token) and
                    result and
                    not result[-1].isspace()):
                    result += ' '
                result += token
                if (any(c.isascii() for c in token) and
                    i < len(cleaned_tokens) - 1):
                    result += ' '

            result = result.strip()
            if result.lower().startswith("what "):
                result = result.replace("what", "什么", 1)

            return result.strip()

        except Exception as e:
            print(f"Error in code-mixing generation: {e}")
            return zh_text

def process_parallel_corpus(df: pd.DataFrame) -> List[str]:
    # Initialize components
    feature_extractor = FeatureExtractor()
    generator = CodeMixedGenerator(feature_extractor)

    code_mixed_texts = []

    for idx, row in df.iterrows():
        try:
            en_text = str(row['instruction_en'])
            zh_text = str(row['instruction_zh'])
            alignment_dict = row['alignment_dict']

            code_mixed = generator.generate_code_mixed(en_text, zh_text, alignment_dict)
            code_mixed_texts.append(code_mixed)

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            code_mixed_texts.append(zh_text)

    return code_mixed_texts

In [ ]:
aligned_df = pd.read_csv('csw_aligned.csv')

In [ ]:
feature_extractor = FeatureExtractor()
generator = CodeMixedGenerator(feature_extractor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
csw_aligned_df = aligned_df.copy()
code_mixed_texts = process_parallel_corpus(csw_aligned_df)
csw_aligned_df['code_mixed'] = code_mixed_texts

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification 

In [ ]:
import random
sample_indices = random.sample(range(len(csw_aligned_df)), 5)
for i in sample_indices:
    row = csw_aligned_df.iloc[i]
    print(f"\nRow {i}:")
    print(f"Chinese: {row['instruction_zh']}")
    print(f"English: {row['instruction_en']}")
    print(f"Alignment_info: {row['alignment_info']}")
    print(f"Code-mixed: {row['code_mixed']}")
    print("-" * 50)


Row 335:
Chinese: 什么是操作系统中的进程调度？
English: What's the process schedule in the operating system?
Alignment_info: {'zh_words': ['什么', '是', '操作系统', '中', '的', '进程', '调度', '？'], 'en_words': ["What's", 'the', 'process', 'schedule', 'in', 'the', 'operating', 'system?'], 'alignment': [(3, 4), (2, 7), (0, 0), (7, 7), (1, 1), (2, 6), (1, 0), (6, 3), (5, 2)]}
Code-mixed: 什么是 operating in 的进程调度？
--------------------------------------------------

Row 298:
Chinese: 什么是区块链的测试网？
English: What's a test network for block chains?
Alignment_info: {'zh_words': ['什么', '是', '区块', '链', '的', '测试', '网', '？'], 'en_words': ["What's", 'a', 'test', 'network', 'for', 'block', 'chains?'], 'alignment': [(4, 4), (0, 0), (1, 1), (6, 3), (7, 6), (3, 6), (2, 5), (5, 2)]}
Code-mixed: 什么 是区块链 for 测试网？
--------------------------------------------------

Row 350:
Chinese: 什么是操作系统中的虚拟内存？
English: What's a virtual memory in an operating system?
Alignment_info: {'zh_words': ['什么', '是', '操作系统', '中', '的', '虚拟内存', '？'], 'en_words'

In [ ]:
csw_aligned_df.to_csv('csw_aligned.csv',index=False)